In [1]:
# In[]
import sys, os
sys.path.append('../')
sys.path.append('../src/')

from unioncom import UnionCom
from test_unioncom_acc import lsi_ATAC

import numpy as np
import pandas as pd
from scipy import sparse
import networkx as nx

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import time

from sklearn.decomposition import PCA
from sklearn.manifold import MDS

import benchmark as bmk
# import de_analy as de

from umap import UMAP

import utils as utils

# import post_align as palign
from scipy.sparse import load_npz

import scanpy as sc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
counts_rna = sc.read("/data1/xcx/unpaired-data/uterus/RNA/highvar_counts_rna.mtx", cache = True).X.toarray()
counts_atac = sc.read("/data1/xcx/unpaired-data/uterus/ATAC/uterus_Zhang2021/highvar_counts_atac.mtx", cache = True).X.toarray()

label_rna = sc.read("/data1/xcx/unpaired-data/uterus/RNA/Uterus_Wang_2020_selected.h5ad", cache = True).obs["cell_type"].values

(8314, 32524)
<class 'numpy.ndarray'>
[[-0.34082285 -0.16343422 -0.01924491 ... -0.3437808  -0.13363136
  -0.7478793 ]
 [-0.34082285 -0.16343422 -0.01924491 ... -0.3437808   0.2874385
   0.00659533]
 [-0.34082285 -0.16343422 -0.01924491 ... -0.3437808   0.25557947
  -0.01994895]
 ...
 [-0.34082285 -0.16343422 -0.01924491 ... -0.3437808  -0.34311333
  -4.035301  ]
 [-0.34082285 -0.16343422 -0.01924491 ... -0.3437808   1.0165334
   0.9535727 ]
 [-0.34082285 -0.16343422 -0.01924491 ... -0.3437808  -1.162309
  -1.3343065 ]]


In [3]:
# dimension reduction
counts_atac = np.where(counts_atac < 1, 0, 1)
counts_atac = lsi_ATAC(counts_atac, k = 1000)

In [5]:
results_dir = "/home/xcx/results/uterus/2-Union-com/"

import datetime
starttime = datetime.datetime.now()

uc = UnionCom.UnionCom(epoch_pd = 3000)
integrated_data = uc.fit_transform([counts_rna, counts_atac])
z_rna = integrated_data[0]
z_atac = integrated_data[1]

#long running
endtime = datetime.datetime.now()
print((endtime-starttime).seconds)

use random seed: 666
Shape of Raw data
Dataset 0: (8237, 1000)
Dataset 1: (8314, 999)
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
use device: cuda:0
epoch:[100/3000] err:1048.7764 alpha:941.4607
epoch:[200/3000] err:140.5151 alpha:126.1369
epoch:[300/3000] err:18.8262 alpha:16.8998
epoch:[400/3000] err:2.5223 alpha:2.2642
epoch:[500/3000] err:0.3379 alpha:0.3034
epoch:[600/3000] err:0.0453 alpha:0.0406
epoch:[700/3000] err:0.0061 alpha:0.0054
epoch:[800/3000] err:0.0008 alpha:0.0007
epoch:[900/3000] err:0.0001 alpha:0.0001
epoch:[1000/3000] err:0.0000 alpha:0.0000
epoch:[1100/3000] err:0.0000 alpha:0.0000
epoch:[1200/3000] err:0.0000 alpha:0.0000
epoch:[1300/3000] err:0.0000 alpha:0.0000
epoch:[1400/3000] err:0.0000 alpha:0.0000
epoch:[1500/3000] err:0.0000 alpha:0.0000
epoch:[1600/3000] err:0.0000 alpha:0.0000
epoch:[1700/3000] err:0.0000 alpha:0.0000
epoch:[1800/3000] err:0.0000 alpha:0.0000
epoch:[1900/3000] err:0.0000 alpha:0.0000
epoch:[20

In [6]:
df = pd.DataFrame(data=z_rna)
df.to_csv(os.path.join("/home/xcx/results/uterus/2-Union-com/z_rna-3000.csv"), index=False)
df = pd.DataFrame(data=z_atac)
df.to_csv(os.path.join("/home/xcx/results/uterus/2-Union-com/z_atac-3000.csv"), index=False)